# **Pendeteksian Outlier dalam Data dengan Metode Local Outlier Factor (LOF)**

## APA ITU LOF
Local Outlier Factor (LOF) adalah metode yang digunakan untuk mengidentifikasi anomali atau outlier dalam suatu dataset dengan membandingkan kepadatan suatu titik terhadap kepadatan titik-titik di sekitarnya. Jika suatu titik memiliki kepadatan yang jauh lebih rendah dibandingkan dengan tetangga-tetangganya, maka kemungkinan besar titik tersebut adalah outlier.

## BAGAIMANA TAHAPAN LOF
Tahapan dari metode **Local Outlier Factor (LOF)** dalam mendeteksi outlier adalah sebagai berikut:

1. **Menentukan Tetangga Terdekat**
   LOF mencari sejumlah tetangga terdekat (k-nearest neighbors) dari setiap titik data dalam dataset.

2. **Menghitung Kepadatan Relatif**
   Untuk setiap titik, dihitung kepadatan lokalnya berdasarkan jarak ke tetangga-tetangga terdekat. Jika titik tersebut berada di daerah dengan kepadatan rendah dibandingkan dengan tetangganya, maka ada indikasi bahwa itu adalah outlier.

3. **Menentukan Nilai LOF**
   LOF menghasilkan skor untuk setiap titik yang mencerminkan seberapa besar titik tersebut menyimpang dari kepadatan normal sekitarnya. Jika nilai LOF jauh lebih besar dari 1, maka titik tersebut cenderung menjadi outlier.

   
## CONTOH MENGHITUNG MANUAL LOF
Untuk memahami perhitungan Local Outlier Factor (LOF) secara manual, kita akan melalui langkah-langkah berikut dengan contoh sederhana.

1. **Dataset**:  
   Mari kita lakukan perhitungan Local Outlier Factor (LOF) secara manual berdasarkan dataset berikut::

| ID | X        | Y        |
|----|----------|----------|
| 1  | 1        | 2        |
| 2  | 2        | 1        |
| 3  | 2        | 3        |
| 4  | 3        | 2        |
| 5  | 10       | 10       |

  Dalam perhitungan ini, kita akan menggunakan k = 2 (dua tetangga terdekat) untuk menghitung LOF.

2. **Tentukan Tetangga Terdekat (k-Nearest Neighbors)**:  
   Kita menggunakan jarak Euclidean untuk menemukan dua tetangga terdekat bagi setiap titik.
   Hasil Tetangga Terdekat:
- (1,2) → Tetangga: (2,1) dan (2,3)
- (2,1) → Tetangga: (1,2) dan (3,2)
- (2,3) → Tetangga: (1,2) dan (3,2)
- (3,2) → Tetangga: (2,1) dan (2,3)
- (10,10) → Tetangga: (3,2) dan (2,3)
  

3. **Hitung Reachability Distance (RD)**   
   RD k(A,B)=max(k-distance(B),d(A,B))
   - k-distance(B) adalah jarak ke tetangga terdekat ke-k.
   - d(A,B) adalah jarak Euclidean antara titik A dan B.

   Contoh Perhitungan:
   - Misal jarak antara (10,10) dan (3,2) adalah besar (~10)
   - k-distance dari (3,2) ke tetangga terdekatnya sekitar 1.41
   - Maka: RD(10,10,3,2)=max(1.41,10)=10


4. **Hitung LOF**:  
LOF(A)= ∑ LRD(A)LRD(B)/k
- Jika LOF mendekati 1, titik tersebut normal.
- Jika LOF jauh lebih besar dari 1, titik tersebut adalah outlier.

Berdasarkan perhitungan ini, nilai LOF untuk (10,10) akan jauh lebih tinggi dibandingkan titik lain, menunjukkan bahwa titik tersebut outlier.



   Semakin kecil nilai LRD, semakin jauh titik dari kepadatannya. Misalnya, jika
RD(10,10) jauh lebih besar dari yang lain, maka kepadatannya rendah.


## IMPLEMENTASI PAKAI SKLEARN UNTUK DATA CONTOH

In [3]:
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

# Dataset contoh
data = {
    'Feature1': [1.0, 2.0, 2.0, 3.0, 10.0],
    'Feature2': [2.0, 1.0, 3.0, 2.0, 10.0]
}

# Membuat DataFrame
df = pd.DataFrame(data)

# Inisialisasi model LOF dengan k=2 (2 tetangga terdekat)
lof = LocalOutlierFactor(n_neighbors=2)

# Fit model LOF dan prediksi label (1 untuk normal, -1 untuk outlier)
lof_labels = lof.fit_predict(df)

# Menambahkan hasil prediksi ke DataFrame
df['LOF Label'] = lof_labels

# Menampilkan hasil
print(df)

# Menampilkan jumlah outlier
num_outliers = (lof_labels == -1).sum()
print(f"\nJumlah outlier: {num_outliers}")

# Menampilkan data outlier
outliers = df[df['LOF Label'] == -1]
print("\nData Outlier:")
print(outliers)


   Feature1  Feature2  LOF Label
0       1.0       2.0          1
1       2.0       1.0          1
2       2.0       3.0          1
3       3.0       2.0          1
4      10.0      10.0         -1

Jumlah outlier: 1

Data Outlier:
   Feature1  Feature2  LOF Label
4      10.0      10.0         -1


In [1]:
%pip install pymysql
%pip install psycopg2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00


Perintah %pip install pymysql dan %pip install psycopg2 digunakan untuk menginstal pustaka yang memungkinkan Python berkomunikasi dengan database MySQL (pymysql) dan PostgreSQL (psycopg2). Dengan pustaka ini, Python dapat menjalankan perintah SQL untuk mengakses, mengelola, dan memanipulasi data dalam database tersebut

In [4]:
import psycopg2
import pymysql
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

def get_pg_data():
    conn = psycopg2.connect(
        host="pg-1c79828e-posgressqlpendata.i.aivencloud.com",
        user="avnadmin",
        password="AVNS_lNd8P_-IyQzpcnKg3Ye",
        database="defaultdb",
        port=14572
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM bunga")
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    cursor.close()
    conn.close()
    return pd.DataFrame(data, columns=columns)

def get_mysql_data():
    conn = pymysql.connect(
        host="mysql-3f95b8aa-mysqll.g.aivencloud.com",
        user="avnadmin",
        password="AVNS_DYTBfDjLFuF2XVSXIqF",
        database="flowers_mysql",
        port=12288
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM flowermysql")
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    cursor.close()
    conn.close()
    return pd.DataFrame(data, columns=columns)

# Ambil data dari kedua database
df_postgresql = get_pg_data()
df_mysql = get_mysql_data()

# Gabungkan berdasarkan kolom 'id' dan 'Class'
df_merged = pd.merge(df_mysql, df_postgresql, on=["id", "class"], how="inner")

# Ambil data fitur numerik
feature_columns = ["petal_length", "petal_width", "sepal_length", "sepal_width"]
data_values = df_merged[feature_columns].values

Kode diatas hanya untuk menggabungkan data dari 2 database saja

In [5]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

# Gabungkan berdasarkan kolom 'id' dan 'class'
df_merge = pd.merge(df_mysql, df_postgresql, on=["id", "class"], how="inner")

# Ambil data fitur numerik tanpa kolom 'class'
feature_columns = ["petal_length", "petal_width", "sepal_length", "sepal_width"]
data_values = df_merge[feature_columns].values

# Inisialisasi model LOF
clf = LocalOutlierFactor(n_neighbors=90)
label = clf.fit_predict(data_values)

# Tambahkan hasil label ke dataframe
df_merge["outlier_label"] = label

# Cetak hasil dengan ID dan class
print(df_merge.to_string(index=False))

num_outliers = (label == -1).sum()
print(f"\nJumlah outlier: {num_outliers}")

outliers = df_merge[df_merge["outlier_label"] == -1]
print("\nData Outlier:")
print(outliers.to_string(index=False))

 id           class  petal_length  petal_width  sepal_length  sepal_width  outlier_label
  1     Iris-setosa           1.4          0.2           5.1          3.5              1
  2     Iris-setosa          14.0          2.0          40.9         30.0             -1
  3     Iris-setosa           1.3          0.2           4.7          3.2              1
  4     Iris-setosa           1.5          0.2           4.6          3.1              1
  5     Iris-setosa           1.4          0.2           5.0          3.6              1
  6     Iris-setosa           1.7          0.4           5.4          3.9              1
  7     Iris-setosa           1.4          0.3           4.6          3.4              1
  8     Iris-setosa           1.5          0.2           5.0          3.4              1
  9     Iris-setosa           1.4          0.2           4.4          2.9              1
 10     Iris-setosa           1.5          0.1           4.9          3.1              1
 11     Iris-setosa  

Kode ini menggabungkan data dari MySQL dan PostgreSQL berdasarkan kolom "id" dan "class" menggunakan metode inner join. Setelah data digabungkan, hanya kolom numerik tertentu yang digunakan sebagai fitur untuk analisis, yaitu "petal_length", "petal_width", "sepal_length", dan "sepal_width".

Selanjutnya, algoritma Local Outlier Factor (LOF) diterapkan dengan 90 tetangga terdekat untuk mendeteksi data yang memiliki kepadatan rendah dibandingkan dengan titik-titik di sekitarnya. Data yang terdeteksi sebagai outlier diberi label -1, sedangkan data normal diberi label 1.

Hasil analisis ini kemudian ditambahkan kembali ke dalam dataframe untuk mempermudah interpretasi. Setelah itu, jumlah outlier dihitung dan ditampilkan, serta semua data yang terdeteksi sebagai outlier dicetak untuk dianalisis lebih lanjut. Dengan pendekatan ini, dataset dapat dibersihkan atau dianalisis lebih mendalam untuk memahami pola anomali yang ada.